In [1]:
from astropy import units as u
u.imperial.enable()

In [2]:
import numpy as np
from numpy import array

In [2]:
dist = 10*u.m

In [3]:
time = 2*u.second

In [5]:
speed = dist/time

In [6]:
speed

<Quantity 5. m / s>

In [7]:
type(speed)

astropy.units.quantity.Quantity

In [15]:
speed.to(u.imperial.inch/u.minute)

<Quantity 11811.02362205 inch / min>

In [44]:
l = array([10,12,14])

In [45]:
l = l * u.m

In [46]:
l

<Quantity [10., 12., 14.] m>

In [47]:
type(l)

astropy.units.quantity.Quantity

In [48]:
l.value

array([10., 12., 14.])

In [21]:
s = "m"

In [22]:
u.Unit(s)

Unit("m")

In [39]:
s = "10 * V * cm^2"

In [40]:
u.Unit(s)

Unit("10 cm2 V")

In [63]:
u1 = u.mW/(u.cm**2)/u.sr/u.nm
#u2 = u.mW/(u.cm**2)/u.nm
u1

Unit("mW / (cm2 nm sr)")

In [65]:
u2 = u.mW/u.sr/u.nm/(u.cm**2)
u2

Unit("mW / (cm2 nm sr)")

In [29]:
a1 = 10*u1

In [30]:
a1.to(u2)

UnitConversionError: 'mW / (cm2 nm sr)' and 'mW / (cm2 nm)' (spectral flux density wav) are not convertible

In [97]:
class uarray(np.ndarray):
    """ Unit-aware array based on Pint

    Example
    -------

    >>> a = uarray(np.linspace(10, 100, 10), 'Td')

    """

    def __new__(cls, input_array, unit=None):
        # Input array is an already formed ndarray instance
        # We first cast to be our class type

        obj = np.asarray(input_array).view(cls)
        # add the new attribute to the created instance

        if unit is not None:
            obj = obj * u.Unit(unit)

        # Finally, we must return the newly created object:
        return obj

    def __array_finalize__(self, obj):
        # see InfoArray.__array_finalize__ for comments
        if obj is None:
            return


#        self.info = getattr(obj, 'info', None)


In [98]:
def conv2(quantity, fromunit, tounit):
    """ Converts `quantity` from unit `fromunit` to unit `tounit`


    Parameters    
    ----------

    quantity: array
        quantity to convert

    fromunit: str
        input unit

    tounit: str
        output unit

    """

    try:
        a = quantity * u.Unit(fromunit)
        a = a.to(u.Unit(tounit))

    except u.UnitConversionError:
        raise TypeError("Cannot convert quantity to the specified unit. Please check the dimensions.")

    return a.value

In [99]:
a = uarray(np.linspace(10, 100, 10), "1e-17 * V * cm**2")
res = np.linspace(1e-16, 1e-15, 10)* u.Unit("V * cm^2")  # pint definition

assert (
    np.round(np.array(a.to("V * cm^2")) - np.array(res), 5) == np.zeros_like(res)
).all()
#    b = (print(a.to('V * cm^2'))==print(res))

# Test conversion
convtable = [
    (500, "nm", 0.5, "µm"),
    (1, "erg/s", 1e-7, "W"),
    (1, "m2", 10000, "cm2"),
]
for a, f, r, t in convtable:
    cr = conv2(a, f, t)
    print(("{0} {1} = {2} {3}".format(a, f, cr, t)))
    assert np.isclose(cr, r)

500 nm = 0.5 µm
1 erg/s = 1.0000000000000001e-07 W
1 m2 = 10000.0 cm2


In [21]:
def is_homogeneous(unit1, unit2):
    """ Tells if unit1 and unit2 are homogeneous, using the Pint library


    Parameters    
    ----------

    unit1, unit2: str
        units 

    """

    try:
        1*u.Unit(unit1) + 1*u.Unit(unit2)
        return True
    except u.UnitConversionError:
        return False


In [22]:
assert not is_homogeneous("mW/cm2/sr/nm", "mW/cm2/nm")

In [61]:
is_homogeneous("mW/cm**2/sr", "W/sr/cm^2")

True

In [95]:
def convert_emi2cm(j_nm, wavenum, Iunit0, Iunit):
    """
    Convert spectral emission density in wavelength base (typically ~mW/cm3/sr/nm) 
    to spectral emission density in wavenumber base (~mW/cm3/sr/cm_1)


    Parameters    
    ----------

    j_nm: array
        spectral emission density in Iunit0 unit (~ per wavelength) )

    wavenum: array (cm-1)
        wavenumber

    Iunit0: str
        unit (~ per wavelength) to convert from 

    Iunit: str
        unit (~ per wavenumber) to convert to

    """

    #not necessary as Astropy does not differentiate between `mw/cm3/sr/nm` and `mW/sr/cm3/nm`
    # if Q_(Iunit0) != Q_(
    #     "mW/cm3/sr/nm"
    # ):  # Q_ makes sure 'mW/sr/cm3/nm' == 'mW/cm3/sr/nm'
    j_nm = conv2(j_nm, Iunit0, "mW/cm3/sr/nm")

    # Convert ''mW/cm3/sr/nm' to 'mW/cm3/sr/cm_1'
    j_cm = j_nm * 1e7 / wavenum ** 2
    # note that we discard the - sign.
    # This means that one of the integrals `trapz(j_nm, wavelen)` or
    # `trapz(j_cm, wavenum)` will be < 0 as they are sorted in opposite order.
    # But both plots look alright

    # Convert to whatever was wanted
    j_cm = conv2(j_cm, "mW/cm**3/sr/cm-1", Iunit)

    return j_cm

In [ ]:
sx1 = "cm**2*mW/cm**3/sr/cm-1"
sx2 = ""

In [96]:
j = [1,2,3]
print(convert_emi2cm(j, 10, "mW/cm**3/sr/nm", "mW/cm**3/sr/cm-1"))

[100000. 200000. 300000.]


In [79]:
s1= "mW/cm3/sr/nm"
s2 = "mW/cm3/sr/cm-1"

In [80]:
x = 10*u.Unit(s2)
x

<Quantity 10. mW / (cm2 sr)>

In [84]:
j_nm = [1, 2, 3]
Iunit0 = "mW/cm^3/sr/nm"
j_nm = conv2(j_nm, Iunit0, "mW/cm^3/sr/nm")

In [85]:
wavenum = 10 
j_cm = j_nm * 1e7 / wavenum ** 2

In [86]:
j_cm

array([100000., 200000., 300000.])

In [ ]:
j_cm = conv2(j_cm, "mW/cm^3/sr/cm-1", "mW/cm")

In [3]:
s1 = "mW/cm2/sr/nm"

In [4]:
u.Unit(s1)

Unit("mW / (cm2 nm sr)")

In [19]:
u.Unit("cm*1")

ValueError: 'cm*1' did not parse as unit: Syntax error parsing unit 'cm*1' If this is meant to be a custom unit, define it with 'u.def_unit'. To have it recognized inside a file reader or other code, enable it with 'u.add_enabled_units'. For details, see http://docs.astropy.org/en/latest/units/combining_and_defining.html

In [20]:
x = 1 * u.Unit("cm-2")
print(x)
x = x.to(u.Unit("m-2"))
print(x)

1.0 1 / cm2
10000.0 1 / m2


<Quantity 1. cm3>

In [8]:
x.to(u.Unit("m3"))

<Quantity 1.e-06 m3>

In [10]:
10 * u.Unit("cm-1")

<Quantity 10. 1 / cm>

In [4]:
x = 10 * u.Unit("m")

In [5]:
x.unit

Unit("m")

In [6]:
Q_ = u.Unit

In [28]:
new_unit = "nm*mW/cm2/sr/µm"

In [27]:
nu = Q_(new_unit)

ValueError: '(mW/cm2/sr/µm)*nm' did not parse as unit: Syntax error parsing unit '(mW/cm2/sr/µm)*nm' If this is meant to be a custom unit, define it with 'u.def_unit'. To have it recognized inside a file reader or other code, enable it with 'u.add_enabled_units'. For details, see http://docs.astropy.org/en/latest/units/combining_and_defining.html

In [21]:
st = str(nu)

In [22]:
mu = Q_(st)

In [23]:
mu

Unit("mW / (cm2 nm sr um)")

In [17]:
t = "{:~P}".format(apple)

ValueError: Invalid format specifier

In [8]:
new_unit = "{:~P}".format(Q_(new_unit).unit)

AttributeError: 'CompositeUnit' object has no attribute 'unit'

In [3]:
from radis.test.utils import getTestFile
from radis import load_spec
s = load_spec("CO_Tgas1500K_mole_fraction0.01.spec")

Could not open file CO_Tgas1500K_mole_fraction0.01.spec with binary=False. Trying with binary=True


FileNotFoundError: [Errno 2] No such file or directory: 'CO_Tgas1500K_mole_fraction0.01.spec'

In [10]:
ls

calc_N2C_spectrum_Trot1200_Tvib3000_slit0.1.txt
calc_N2C_spectrum_Trot1200_Tvib3000.txt
cdsd_hitemp_09_fragment.h5
cdsd_hitemp_09_fragment.txt
cdsd_hitemp_09_header.h5
cdsd_hitemp_09_header.txt
CO2abscoeff_300K_4150_4400nm.txt
co2_cdsd_hamiltonian_fragment.levels
co2_cdsd_hamiltonian_fragment.levels.h5
CO_Tgas1500K_mole_fraction0.01.spec
CO_Tgas1500K_mole_fraction0.5.spec
generate_test_files.py
hitran_2016_H2O_2iso_2000_2100cm.h5
hitran_2016_H2O_2iso_2000_2100cm.par
hitran_co2_626_bandhead_4165_4200nm.h5
hitran_co2_626_bandhead_4165_4200nm.par
hitran_CO2_fragment.h5
hitran_CO2_fragment.par
hitran_co_3iso_2000_2300cm.h5
hitran_co_3iso_2000_2300cm.par
hitran_CO_fragment.h5
hitran_CO_fragment.par
N2C_specair_380nm.spec
slitfunction.txt


In [18]:
from radis.test.utils import getTestFile
from radis import load_spec
load_spec("CO_Tgas1500K_mole_fraction0.01.spec").plot()

Could not open file CO_Tgas1500K_mole_fraction0.01.spec with binary=False. Trying with binary=True
Worked! Use binary=True directly in load_spec for faster loading
File CO_Tgas1500K_mole_fraction0.01.spec has a deprecrated structure (key medium removed in 0.9.22). Fixing this time, but regenerate database ASAP.
File CO_Tgas1500K_mole_fraction0.01.spec has a deprecrated structure (v1u in lines is now called vu). Fixed this time, but regenerate database ASAP.


ValueError: 
^\mathregular{-I/I
                  ^
Expected {snowflake | accent | symbol | c_over_c | function | group | frac | dfrac | stackrel | binom | genfrac | sqrt | overline | operatorname}, found end of text  (at char 18), (line:1, col:19)

ValueError: 
^\mathregular{-I/I
                  ^
Expected {snowflake | accent | symbol | c_over_c | function | group | frac | dfrac | stackrel | binom | genfrac | sqrt | overline | operatorname}, found end of text  (at char 18), (line:1, col:19)

<Figure size 640x480 with 1 Axes>